In [1]:
!source my_env/bin/activate

In [1]:
# Install the packages
! pip install --upgrade --quiet google-cloud-aiplatform
! pip install google-cloud-discoveryengine

In [2]:
PROJECT_ID = "cacafly-ml-specialization"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

In [3]:
import vertexai
from vertexai.language_models import CodeGenerationModel

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [4]:
import sys
import json
import os
import vertexai
from typing import Dict, List, Optional, Tuple
from google.cloud import discoveryengine
from google.protobuf.json_format import MessageToDict

In [5]:
search_engine_id = "demo3_1733212957986"
serving_config_id = "default_config"

In [6]:
# !pip show numpy
# !pip show pandas
# !pip install -U "scikit-learn" --force-reinstall


In [7]:

import tarfile

import numpy as np 
import pandas as pd 
from sklearn.datasets import load_files
import nltk 
from nltk.corpus import stopwords

In [8]:
movie_data = load_files('txt_sentoken/')
X , y = movie_data.data , movie_data.target

In [18]:
import json

with open("input_with_name_comment.jsonl", "w", encoding="utf-8") as f:
    for i, text in enumerate(X, start=1):
        prompt = (
            "Please remove all names of people and works of art (like movies, books, songs) "
            "from the following text. Keep grammar natural.\n\n"
            f"Original:\n{text}\nCleaned:"
        )
        json_line = {
            "id": i,
            "request": {
                "contents": [
                    {
                        "parts": {
                            "text": prompt
                        },
                        "role": "user"
                    }
                ]
            }
        }
        json.dump(json_line, f, ensure_ascii=False)
        f.write("\n")


In [19]:
import time

from google import genai
from google.genai.types import CreateBatchJobConfig, JobState, HttpOptions

# client = genai.Client(http_options=HttpOptions(api_version="v1"))
client = genai.Client(
    vertexai=True,  # ✅ 告訴它你是用 Vertex AI 而不是 api_key
    project="cacafly-ml-specialization",        # ✅ GCP 專案 ID
    location="us-central1",               # ✅ Vertex AI 支援的地區
    http_options=HttpOptions(api_version="v1")
)
# TODO(developer): Update and un-comment below line
output_uri = "gs://cacafly-ml-specialization-dataset/movie_review/"

# See the documentation: https://googleapis.github.io/python-genai/genai.html#genai.batches.Batches.create
job = client.batches.create(
    model="gemini-2.0-flash-001",
    # Source link: https://storage.cloud.google.com/cloud-samples-data/batch/prompt_for_batch_gemini_predict.jsonl
    src="gs://cacafly-ml-specialization-dataset/movie_review/input_with_name_comment.jsonl",
    # src="gs://cloud-samples-data/batch/prompt_for_batch_gemini_predict.jsonl",
    config=CreateBatchJobConfig(dest=output_uri),
)
print(f"Job name: {job.name}")
print(f"Job state: {job.state}")
# Example response:
# Job name: projects/%PROJECT_ID%/locations/us-central1/batchPredictionJobs/9876453210000000000
# Job state: JOB_STATE_PENDING

# See the documentation: https://googleapis.github.io/python-genai/genai.html#genai.types.BatchJob
completed_states = {
    JobState.JOB_STATE_SUCCEEDED,
    JobState.JOB_STATE_FAILED,
    JobState.JOB_STATE_CANCELLED,
    JobState.JOB_STATE_PAUSED,
}

while job.state not in completed_states:
    time.sleep(30)
    job = client.batches.get(name=job.name)
    print(f"Job state: {job.state}")
# Example response:
# Job state: JOB_STATE_PENDING
# Job state: JOB_STATE_RUNNING
# Job state: JOB_STATE_RUNNING
# ...
# Job state: JOB_STATE_SUCCEEDED

Job name: projects/765271398193/locations/us-central1/batchPredictionJobs/7164113081608437760
Job state: JOB_STATE_PENDING
Job state: JOB_STATE_RUNNING
Job state: JOB_STATE_RUNNING
Job state: JOB_STATE_RUNNING
Job state: JOB_STATE_RUNNING
Job state: JOB_STATE_RUNNING
Job state: JOB_STATE_RUNNING
Job state: JOB_STATE_RUNNING
Job state: JOB_STATE_RUNNING
Job state: JOB_STATE_RUNNING
Job state: JOB_STATE_SUCCEEDED
